In [ ]:
import sqlite3 
import pandas as pd
import numpy as np
import seaborn as sns
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import missingno as msno

from sklearn.ensemble import IsolationForest

In [ ]:

# Connect to the database (or create it if it doesn't exist)
conn = sqlite3.connect('car_listings.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Step 1: Update the make column
cursor.execute("""
    UPDATE car_listings
    SET 
        make = CASE 
            WHEN make = 'Aston' THEN 'Aston Martin'
            WHEN make = 'Alfa' THEN 'Alfa Romeo'
            WHEN make = 'Land' THEN 'Land Rover'
ELSE make
        END,
        model = CASE
            WHEN make = 'Aston' AND model LIKE 'Martin %' THEN SUBSTR(model, INSTR(model, 'Martin ') + 7)
            WHEN make = 'Alfa' AND model LIKE 'Romeo %' THEN SUBSTR(model, INSTR(model, 'Romeo ') + 6)
            WHEN make = 'Land' AND model LIKE 'Rover %' THEN SUBSTR(model, INSTR(model, 'Rover ') + 6)
            ELSE model
        END
""")

# Commit the changes to the database
conn.commit()

# Close the connection
conn.close()

In [ ]:
import sqlite3
import csv

# Connect to the SQLite database
conn = sqlite3.connect('car_listings.db')
cursor = conn.cursor()

# Execute a query to retrieve all data from the desired table
cursor.execute("SELECT * FROM car_listings")
rows = cursor.fetchall()

# Define the CSV file name
csv_filename = 'car_listings.csv'

# Write data to a CSV file
with open(csv_filename, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write the header (column names)
    csv_writer.writerow([description[0] for description in cursor.description])
    # Write the data rows
    csv_writer.writerows(rows)

# Close the SQLite connection
conn.close()


In [ ]:
cars_db = pd.read_csv("car_listings.csv")

In [ ]:
# Checking the columns present in the data
cars_db.columns

In [ ]:
# Checking the first 5 rows of the dataset
cars_db.head()

In [ ]:
# Understand the shape of the data
cars_db.shape

In [ ]:
# Extracting basic info about the dataframe.
cars_db.info()

In [ ]:
# checking if the data has null values present.
cars_db.isnull().sum()

In [ ]:
# Checking the Dataframe for duplicate values in each column.
list_columns = cars_db.columns

for column in list_columns:
    number_of_duplicates = cars_db[column].duplicated().sum()
    print(f'Number of duplicates in column {column}: {number_of_duplicates}')

In [ ]:
# Checking the count of unique values in each column
cars_db.nunique()

In [ ]:
# calling a describe method to get some insight on different aspects of the data.
cars_db.describe(include='all').T

In [ ]:
# getting some of the same info as the .descrive function for the categorical values.
cars_db.describe(include=object).T

In [ ]:
# Drop the 'timestamp' column
cars_db_1 = cars_db.drop(columns='timestamp')

In [ ]:

fig, axes = plt.subplots(3, 1, figsize=(15, 20))
fig.suptitle('Visualization for key variables in the dataset')

plot_titles = ['Count of Makes', 'Mileage Histogram', 'Listing Price Histogram']
plot_features = ['make', 'mileage', 'listing_price']

for i, ax in enumerate(axes.flatten()):
    feature = plot_features[i]
    if feature in ['make']:
        sns.countplot(y=feature, data=cars_db_1, ax=ax)  # Changed to countplot for categorical data
    else:
        sns.histplot(x=feature, data=cars_db_1, kde=True, ax=ax)
        ax.axvline(cars_db_1[feature].mean(), color="green", linestyle="--")
        ax.axvline(cars_db_1[feature].median(), color="black", linestyle="-")
    ax.set_title(plot_titles[i])

plt.subplots_adjust(top=0.9)  # Adjusts the space to prevent title overlapping
plt.tight_layout()
plt.show()


In [ ]:
# Plotting some boxplots to identify possible outliers.

result = cars_db_1.select_dtypes(include = 'number')

cont_cols = list(result)

for col in cont_cols:

    print(col)
    
    print('Skew :',round(cars_db_1[col].skew(),2))
    
    plt.figure(figsize = (15, 4))
    
    plt.subplot(1, 2, 1)
    
    cars_db_1[col].hist(bins = 10, grid = False)
    
    plt.ylabel('count')
    
    plt.subplot(1, 2, 2)
    
    sns.boxplot(x = cars_db_1[col])
    
    plt.show()

In [ ]:
# finding the 'kilometers_driven' outlier index value
cars_db_1.sort_values('listing_price', ascending=False, na_position='last') 

In [ ]:
# Dropping the extreme outlies
cars_db_2 = cars_db_1.drop(labels=[0,287,535,536], axis=0)

In [ ]:
# finding the 'kilometers_driven' outlier index value
cars_db_2.sort_values('listing_price', ascending=False, na_position='last') 

In [ ]:
# Replotting the histograms for 'lisitng_price' after dropping the outliers

fig, axes = plt.subplots(2, 1, figsize=(12, 14))

# Histogram
sns.histplot(x='listing_price', data=cars_db_2, kde=True, ax=axes[0])
print('Skew :',round(cars_db_2[col].skew(),2))
axes[0].axvline(cars_db_2['listing_price'].mean(), color="green", linestyle="--")
axes[0].axvline(cars_db_2['listing_price'].median(), color="black", linestyle="-")
axes[0].set_xlabel('listing_price')
axes[0].set_ylabel('Frequency')

# Boxplot
sns.boxplot(x='listing_price', data=cars_db_2, ax=axes[1])
axes[1].set_xlabel('listing_price')
axes[1].set_ylabel('')

plt.show()

In [ ]:
# Reseting index in order to update the dropped rows.
cars_db_2 = cars_db_2.reset_index(drop = True)

In [ ]:
cars_db_2['make'].unique()

In [ ]:
# List of makes to filter
selected_makes = ["Toyota", "BMW", "Jeep", "Volvo", "Ford", "Mazda", "Chevrolet", "RAM", "Land Rover", "Lexus"]

# Filter the DataFrame
filtered_cars_db_2 = cars_db_2[cars_db_2['make'].isin(selected_makes)]
print(filtered_cars_db_2.head(60))

In [ ]:
# Find the indices of the rows you want to drop
indices_to_drop = cars_db_2.index[cars_db_2['id'].isin([11, 16, 23, 36, 61, 62, 63, 64, 365, 503, 598, 613, 644, 1057, 1271, 1345, 1603, 1772])]

# Drop these indices from the DataFrame
cars_db_3 = cars_db_2.drop(indices_to_drop)

In [ ]:
# List of makes to filter
selected_makes = ["Toyota", "BMW", "Jeep", "Volvo", "Ford", "Mazda", "Chevrolet", "RAM", "Land Rover", "Dodge"]

# Filter the DataFrame
filtered_cars_db_3 = cars_db_3[cars_db_3['make'].isin(selected_makes)]
print(filtered_cars_db_3.head(50))

In [ ]:
# Reseting index in order to update the dropped rows.
cars_db_3 = cars_db_3.reset_index(drop = True)

cars_db_3['make'].unique()

In [ ]:
cars_db_3

In [ ]:
plt.figure(figsize=(12, 6))

# Scatter plot - cars_df_1
sns.scatterplot(x='mileage', y='listing_price', data=cars_db_3)
plt.title('Scatter Plot')
plt.xlabel('mileage')
plt.ylabel('listing_price')

In [ ]:
# finding the 'Mileage' outlier index value
cars_db_3.sort_values('mileage', ascending=False, na_position='last') 

In [ ]:
# Set the figure size
plt.figure(figsize = (15, 8))

# Create the boxplots
sns.boxplot(x = cars_db_3['make'], y = cars_db_3['listing_price'])
# Rotate the labels on the x-axis to 45 degrees
plt.xticks(rotation=45)

plt.show()

In [ ]:
# List of makes to filter
selected_makes = ["Toyota", "BMW", "Jeep", "Volvo", "Mazda", "Chevrolet", "RAM", "Land Rover", "Dodge", "Hummer"]

# Filter the DataFrame
filtered_cars_db_4 = cars_db_3[cars_db_3['make'].isin(selected_makes)]
print(filtered_cars_db_4.head(50))

In [ ]:
# Saving a new cured CSV
cars_db_3.to_csv('car_listings_cured.csv', index=False)

In [ ]:
# Plottinh a distribution of "Car_Class" in Engine vs Power
plt.figure(figsize=(15,12))
sns.scatterplot(data=cars_db_3,y='listing_price',x='mileage',hue='make')
plt.show()

In [ ]:
cars_db_3['make'].unique()

In [ ]:
import os
import shutil

In [ ]:

# Organize by make - Create Folders
def sort_images_by_brand(data_dir, brands):
    """
    Organizes images in `data_dir` by moving them into subdirectories based on the brand names present in their filenames.

    Args:
    - data_dir (str): The main directory containing all the images.
    - brands (list of str): List of car brand names.
    """
    for file_name in os.listdir(data_dir):
        file_path = os.path.join(data_dir, file_name)
        
        # Only process files (avoid directories)
        if os.path.isfile(file_path):
            for brand in brands:
                if brand.replace(' ', '_') in file_name:
                    brand_dir = os.path.join(data_dir, brand.replace(' ', '_'))
                    
                    # Create brand directory if it doesn't exist
                    if not os.path.exists(brand_dir):
                        os.makedirs(brand_dir)
                    
                    dest_path = os.path.join(brand_dir, file_name)
                    shutil.move(file_path, dest_path)
                    break  # Exit the inner loop once we've found a matching brand and moved the file

# Usage:

data_dir = 'C:\\Users\\franc\\Documents\\GitHub\\Car_Detection_Chat\\images'
brands = ['Ferrari', 'Bugatti', 'Koenigsegg', 'Lamborghini', 'Maserati',
          'McLaren', 'Porsche', 'Maybach', 'Mercedes-Benz', 'Ford', 'Lexus',
          'Aston Martin', 'Rolls-Royce', 'Chevrolet', 'Dodge', 'Cadillac',
          'DeTomaso', 'Bentley', 'Alfa Romeo', 'Jaguar', 'Jeep', 'BMW',
          'Spyker', 'Audi', 'Land Rover', 'Acura', 'Hummer']

sort_images_by_brand(data_dir, brands)

